# Capstone Project 1 - Applicance Energy Prediction

In [135]:
# Import the libraries -

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline
from datetime import datetime as dt
# Seed value for reproducing the code
seed = 79
np.random.seed(seed)

# Load the Dataset for Energy data

In [136]:
df = pd.read_csv('energydata_complete.csv')

In [150]:
df.head(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


# Load the Dataset for Gas price for the period of year

In [137]:
dfoil = pd.read_csv('CrudeOilPrice.csv')

In [151]:
dfoil.head(2)

,date,value
0,2/3/10,76.98
1,2/4/10,73.14


# Dataset Energy Appliance - Statistics and Details

In [138]:
print("Number of instances in dataset = {}".format(df.shape[0]))
print()
print("Total number of columns = {}".format(df.columns.shape[0]))
print()
print("Column wise count of null values:-")
print()
print(df.isnull().sum())
print()
print(df.info())

Number of instances in dataset = 19735

Total number of columns = 29

Column wise count of null values:-

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
date           19735 non-null object
Appliances     19735 non-null int64
lights         19735 non-null int64
T1             19735 non-null float64
RH_1           19735 non-null float64
T2             19735 non-null float64
RH_2           19735 non-null float64
T3             19

# Lets find the date range of the data

In [139]:
print("Data Captured for the period of ", min(df.date), 'and ', max(df.date))

Data Captured for the period of  2016-01-11 17:00:00 and  2016-05-27 18:00:00


# Dataset Gas Price - Statistics and Details

In [140]:
print(dfoil.shape)
print()
print(dfoil.describe())
print()
print(dfoil.info())

(2519, 2)

             value
count  2519.000000
mean     72.352326
std      21.912538
min      26.210000
25%      52.411000
50%      70.250000
75%      93.550000
max     113.930000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2519 entries, 0 to 2518
Data columns (total 2 columns):
date      2519 non-null object
 value    2519 non-null float64
dtypes: float64(1), object(1)
memory usage: 39.4+ KB
None


In [141]:
# Lets parse the columns with respect to its dimensions 

# Temperature sensors columns
temp_cols = ["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9"]

# Humidity sensors columns
hum_cols = ["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9"]

# Weather data columns
wth_cols = ["T_out", "Tdewpoint", "RH_out", "Press_mm_hg", "Windspeed", "Visibility"]

# Target column
tgt = ["Appliances"]

In [142]:
print(df[temp_cols].describe())

                 T1            T2            T3            T4            T5  \
count  19735.000000  19735.000000  19735.000000  19735.000000  19735.000000   
mean      21.686571     20.341219     22.267611     20.855335     19.592106   
std        1.606066      2.192974      2.006111      2.042884      1.844623   
min       16.790000     16.100000     17.200000     15.100000     15.330000   
25%       20.760000     18.790000     20.790000     19.530000     18.277500   
50%       21.600000     20.000000     22.100000     20.666667     19.390000   
75%       22.600000     21.500000     23.290000     22.100000     20.619643   
max       26.260000     29.856667     29.236000     26.200000     25.795000   

                 T6            T7            T8            T9  
count  19735.000000  19735.000000  19735.000000  19735.000000  
mean       7.910939     20.267106     22.029107     19.485828  
std        6.090347      2.109993      1.956162      2.014712  
min       -6.065000     15.39000

In [143]:
print(df[hum_cols].describe())

               RH_1          RH_2          RH_3          RH_4          RH_5  \
count  19735.000000  19735.000000  19735.000000  19735.000000  19735.000000   
mean      40.259739     40.420420     39.242500     39.026904     50.949283   
std        3.979299      4.069813      3.254576      4.341321      9.022034   
min       27.023333     20.463333     28.766667     27.660000     29.815000   
25%       37.333333     37.900000     36.900000     35.530000     45.400000   
50%       39.656667     40.500000     38.530000     38.400000     49.090000   
75%       43.066667     43.260000     41.760000     42.156667     53.663333   
max       63.360000     56.026667     50.163333     51.090000     96.321667   

               RH_6          RH_7          RH_8          RH_9  
count  19735.000000  19735.000000  19735.000000  19735.000000  
mean      54.609083     35.388200     42.936165     41.552401  
std       31.149806      5.114208      5.224361      4.151497  
min        1.000000     23.20000

In [144]:
print(df[wth_cols].describe())

              T_out     Tdewpoint        RH_out   Press_mm_hg     Windspeed  \
count  19735.000000  19735.000000  19735.000000  19735.000000  19735.000000   
mean       7.411665      3.760707     79.750418    755.522602      4.039752   
std        5.317409      4.194648     14.901088      7.399441      2.451221   
min       -5.000000     -6.600000     24.000000    729.300000      0.000000   
25%        3.666667      0.900000     70.333333    750.933333      2.000000   
50%        6.916667      3.433333     83.666667    756.100000      3.666667   
75%       10.408333      6.566667     91.666667    760.933333      5.500000   
max       26.100000     15.500000    100.000000    772.300000     14.000000   

         Visibility  
count  19735.000000  
mean      38.330834  
std       11.794719  
min        1.000000  
25%       29.000000  
50%       40.000000  
75%       40.000000  
max       66.000000  


In [145]:
print(df['lights'].describe())

count    19735.000000
mean         3.801875
std          7.935988
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         70.000000
Name: lights, dtype: float64


In [146]:
print(df[tgt].describe())

         Appliances
count  19735.000000
mean      97.694958
std      102.524891
min       10.000000
25%       50.000000
50%       60.000000
75%      100.000000
max     1080.000000


# Merge df and dfoil

In [160]:
df['dateupdate'] = pd.to_datetime(df['date']).dt.normalize()
df['dateupdate'].head(2)
#df.astype({'dateupdate': 'int32'}).dtypes

0   2016-01-11
1   2016-01-11
Name: dateupdate, dtype: datetime64[ns]

In [ ]:
dfmerge=pd.merge(left = df, right = dfoil, how='left', left_on='dateupdate', right_on='date')

# Merged Dataset details

In [149]:
print(dfmerge.shape)
print()
print(dfmerge.describe())
print()
print(dfmerge.info())

(19735, 30)

         Appliances        lights            T1          RH_1            T2  \
count  19735.000000  19735.000000  19735.000000  19735.000000  19735.000000   
mean      97.694958      3.801875     21.686571     40.259739     20.341219   
std      102.524891      7.935988      1.606066      3.979299      2.192974   
min       10.000000      0.000000     16.790000     27.023333     16.100000   
25%       50.000000      0.000000     20.760000     37.333333     18.790000   
50%       60.000000      0.000000     21.600000     39.656667     20.000000   
75%      100.000000      0.000000     22.600000     43.066667     21.500000   
max     1080.000000     70.000000     26.260000     63.360000     29.856667   

               RH_2            T3          RH_3            T4          RH_4  \
count  19735.000000  19735.000000  19735.000000  19735.000000  19735.000000   
mean      40.420420     22.267611     39.242500     20.855335     39.026904   
std        4.069813      2.006111     